In [7]:
using Logging, Revise
global_logger(ConsoleLogger(stdout, Logging.Info));

In [8]:
using PorousMaterials, MOFun

In [9]:
s_moty = moiety("find-replace/2-!-p-phenylene")
r_moty = moiety("2-acetylamido-p-phenylene")
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent;

In [158]:
function fndrplc(s_moty::Crystal, r_moty::Crystal, parent::Crystal;
        configs::Array{Configuration}=Configuration[], 
        search::Search=Search(),
        kwargs...)::Crystal
    
    # mutation guard
    s_moty, r_moty = deepcopy.([s_moty, r_moty])
    
    # if search not specified, do it now
	search = MOFun.isequal(search, Search()) ? s_moty ∈ parent : search
    # if there are no replacements to be made, just return the parent
    if search.num_isomorphisms == 0
        return parent
    end
    
    # determine configuration index array
    c = Int[]
    if configs == Configuration[] # if configs not specified, choose one randomly
        push!(c, rand(1:search.num_isomorphisms))
    else # find indices of specified configs
        for config in configs
            for i in 1:search.num_isomorphisms
                if search.results[i].configuration == config
                    push!(c, i)
                    break
                end
            end
        end
    end
    
    # determine s_mask (which atoms from s_moty are NOT in r_moty?)
    mask = s_moty[MOFun.idx_filter(s_moty, MOFun.r_group_indices(s_moty))]
    s_mask_atoms = mask.atoms
    
    # shift all r_moty nodes according to center of isomorphic subset
    m2r_isom = (mask ∈ r_moty).results[1].isomorphism
    r_moty.atoms.coords.xf .-= MOFun.geometric_center(r_moty[m2r_isom])
    
    # loop over configs to build replacement data
    species = Symbol[]
    fx = Array{Float64, 2}
    xrms = Crystal[]
    del_atoms = Int[]
    for cᵢ in c
        ## get new atom data
        
        # find isomorphism
        s2p_isom = search.results[cᵢ].isomorphism
        
        # find parent subset
        parent_subset = deepcopy(parent[s2p_isom])
        
        # adjust coordinates for periodic boundaries
        MOFun.adjust_for_pb!(parent_subset)
        
        # record the center of xtal_subset so we can translate back later
        parent_subset_center = MOFun.geometric_center(parent_subset)
        
        # shift to align centers at origin
        MOFun.center_on_self!.([parent_subset, s_moty])
        
        # do orthog. Procrustes for s_moty-to-parent and mask-to-replacement alignments
        rot_s2p = MOFun.s2p_op(s_moty, parent_subset)
        rot_r2m = MOFun.r2m_op(r_moty, s_moty, m2r_isom, s_mask_atoms)
        
        # transform r_moty by rot_r2m, rot_s2p, and xtal_subset_center, align to parent
        # (this is now a crystal to add)
        xrm = MOFun.xform_r_moty(r_moty, rot_r2m, rot_s2p, parent_subset_center, parent)
        push!(xrms, xrm)
        
        # push obsolete atoms to array
        for x in s2p_isom
            push!(del_atoms, x) # this can create duplicates; remove them later
        end
        
        # TODO push new bonds to dictionary
    end
    
    # clean up del_atoms
    del_atoms = unique(del_atoms)
    
    # build new crystal
    name = "TODO" # TODO
    atoms = parent.atoms + sum([xrm.atoms for xrm in xrms])
    xtal = Crystal(name, parent.box, atoms, Charges{Frac}(0))
    
    # TODO append temporary crystals to parent
    
    # TODO create bonds from dictionary
    
    # delete atoms from array
    xtal = xtal[[x for x in 1:xtal.atoms.n if !(x ∈ del_atoms)]]
    
    # return result
    return xtal
end

function fndrplc(s, r, p; config, kwargs...)
    return fndrplc(s, r, p, configs=[config], kwargs...)
end

fndrplc (generic function with 2 methods)

In [159]:
xtal = fndrplc(s_moty, r_moty, parent, search=search, configs=[Configuration(1,1), Configuration(2,1)])

Name: TODO
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 25.832000 Å. b = 25.832000 Å, c = 25.832000 Å
	Volume of unit cell: 17237.492730 Å³

	# atoms = 438
	# charges = 0
	chemical formula: Dict(:N => 1,:Zn => 16,:H => 51,:O => 53,:C => 98)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [153]:
write_cif(xtal, "OUTPUT")